# AB-тесты своими руками

Проделаем историю с Дядей Фёдором руками!

In [1]:
import numpy as np

In [2]:
x_f = np.array([150, 190])
x_s = np.array([180, 190])
x_m = np.array([150, 160])

In [3]:
x_f.mean()

170.0

In [4]:
x_f.size

2

In [5]:
x_f.std(ddof=1) # ddof=1 тк делим на n - 1, если на n, тогда ddof=0

28.284271247461902

In [6]:
from scipy import stats

# так можно найти квантиль - то есть число левее которого alpha% выборки
stats.norm().ppf(0.05)

-1.6448536269514729

In [7]:
# хотим чтобы с каждой сторону купола было по 2.5% (если хотим 95% интервал)
# Уточнили правило 2-х сигм! Чтобы было ровно 95%, надо брать не 2, а 1.96
stats.norm().ppf(1 - 0.05/2)

1.959963984540054

Доверительный интервал для среднего своими руками:

In [8]:
def conf_int(x, alpha=0.05):
    mn = x.mean()                       # нашли среднее
    st = x.std(ddof=1)/np.sqrt(x.size)  # нашли стандартное отклонение 
    
    # нужный нам квантиль, в правиле 3-х сигм, q=3
    q = stats.norm().ppf(1 - alpha/2)
    
    left = mn - q * st
    right = mn + q * st
    
    print(f"Доверительный интервал: [{left};{right}])")
    return left, right

l,r = conf_int(x_f)
r - l # длина интервала

Доверительный интервал: [130.80072030919894;209.19927969080106])


78.39855938160213

Пакетный способ сделать то же самое.

In [10]:
stats.norm.interval(0.95, x_f.mean(), x_f.std(ddof=1)/np.sqrt(x_f.size))

(130.80072030919894, 209.19927969080106)

Теперь поговорим про гипотезы! Проверим гипотезу, что $\mu = 160$.

In [11]:
# alpha = 0.05 

# H_0: mu = 160
# H_a: mu != 160

stats.norm.interval(0.95, x_f.mean() - 160, x_f.std(ddof=1)/np.sqrt(x_f.size))

(-29.199279690801077, 49.19927969080108)

Видим, что ноль попал в доверительный интервал $\Rightarrow$ гипотеза не отвергается. 

Офирмим это в виде функции, чтобы считалось через $t$-статистику.

In [12]:
def mean_norm_test(x, mu0, alpha=0.05):
    mn = x.mean() - mu0
    st = x.std(ddof=1)/np.sqrt(x.size)
    
    t = mn/st # нашли наблюдаемое значение t-статистики
    
    # критическое значение t-статистики 
    q = stats.norm().ppf(1 - alpha/2)
    
    if (t < q) & (t > -q): # np.abs(t) < q
        return "Гипотеза не отвергается"
    else:
        return "Гипотеза отвергается"

In [13]:
mean_norm_test(x_f, mu0=160, alpha=0.05)

'Гипотеза не отвергается'

## Bonus-track для целевой аудитории семинара в Zoom :) 

Поговорим про ошибку первого рода!

In [14]:
norm_rv = stats.norm( ) # инициализровали нормальное распределение N(0, 1)
                        # нулевое мат ожидание, единичное стандартное отклонение

norm_rv.rvs(10) # сгенерировали 10 наблюдений 

array([ 2.46695432,  0.07103873,  0.28820446,  0.75078223,  1.11529013,
        1.54429739,  0.68892519, -1.8845052 ,  2.33624069,  2.37726645])

In [15]:
norm_rv.ppf(0.95) # можно найти квантиль уровня 95%

1.6448536269514722

In [16]:
import matplotlib.pyplot as plt

x = np.linspace(-5, 5, 300)
pdf = norm_rv.pdf(x) # плотность распределения вероятностей

alpha = 0.05 
q = norm_rv.ppf(1 - alpha/2)

plt.plot(x, pdf);
plt.vlines(q, 0, 0.4)
plt.text(q + 0.1, 0.4, f'{q:.3}')

plt.vlines(-q, 0, 0.4)
plt.text(-q + 0.1, 0.4, f'{-q:.3}')

Text(-1.859963984540054, 0.4, '-1.96')

Уровень значимости (ошибка первого рода) - вероятность отвергнуть гипотезу $H_0$, когда она верна. Попробуем с помощью симуляций вычислить её для проверки гипотезы о среднем.

In [17]:
def mean_norm_test(x, mu0, alpha=0.05):
    mn = x.mean() - mu0
    st = x.std(ddof=1)/np.sqrt(x.size)
    
    t = mn/st # нашли наблюдаемое значение t-статистики
    
    # критическое значение t-статистики 
    q = stats.norm().ppf(1 - alpha/2)
    
    if (t < q) & (t > -q): # np.abs(t) < q
        return 1
    else:
        return 0

Генерируем данные из нормального распределения с математическим ожиданием (`loc=4`). И проверяем по сгенерированной выборке гипотезу, что $\mu = 4$. В $5\%$ случаев мы почём-зря отвергнем эту гипотезу.

In [18]:
n_obs = 10**4
itog = np.zeros(n_obs)

for i in range(n_obs):
    # задали генератор
    norm_rv = stats.norm(loc=4, scale=1)
    
    # сгенерировали выборку объёма 100
    x = norm_rv.rvs(100)
    itog[i] = mean_norm_test(x, mu0=4, alpha=0.05)

In [19]:
itog.mean() # как часто не отвергали (95%)

0.9486